In [7]:
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential

from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import KernelFunctionFromPrompt

In [8]:
def _create_kernel_with_chat_completion() -> Kernel:
    load_dotenv()
    
    kernel = Kernel()

    # Create Azure credential
    credential = DefaultAzureCredential()

    # Create a token provider function
    def get_azure_ad_token():
        """Function to get Azure AD token for OpenAI."""
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        return token.token

    # Create an AI Service using AzureChatCompletion with DefaultAzureCredential
    chat_completion_service = AzureChatCompletion(
        deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
        endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
        ad_token_provider=get_azure_ad_token
    )

    kernel.add_service(chat_completion_service)

    return kernel

In [12]:
async def main():
    REVIEWER_NAME = "Concierge"
    REVIEWER_INSTRUCTIONS = """
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """
    agent_reviewer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion(),
        name=REVIEWER_NAME,
        instructions=REVIEWER_INSTRUCTIONS,
    )

    FRONTDESK_NAME = "FrontDesk"
    FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """
    agent_writer = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion(),
        name=FRONTDESK_NAME,
        instructions=FRONTDESK_INSTRUCTIONS,
    )

    termination_function = KernelFunctionFromPrompt(
        function_name="termination",
        prompt="""
        Determine if the recommendation process is complete.
        
        The process is complete when the Concierge provides approval for any recommendation made by the Front Desk.
        Look for phrases like "approved", "this recommendation is approved", or any clear indication that the Concierge is satisfied with the suggestion.
        
        If the Concierge has given approval in their most recent response, respond with: yes
        Otherwise, respond with: no
        
        History:
        {{$history}}
        """,
    )

    selection_function = KernelFunctionFromPrompt(
        function_name="selection",
        prompt=f"""
        Determine which participant takes the next turn in a conversation based on the the most recent participant.
        State only the name of the participant to take the next turn.
        No participant should take more than one turn in a row.
        
        Choose only from these participants:
        - {REVIEWER_NAME}
        - {FRONTDESK_NAME}
        
        Always follow these rules when selecting the next participant, each conversation should be at least 4 turns:
        - After user input, it is {FRONTDESK_NAME}'s turn.
        - After {FRONTDESK_NAME} replies, it is {REVIEWER_NAME}'s turn.
        - After {REVIEWER_NAME} provides feedback, it is {FRONTDESK_NAME}'s turn.

        History:
        {{{{$history}}}}
        """,
    )

    chat = AgentGroupChat(
        agents=[agent_writer, agent_reviewer],
        termination_strategy=KernelFunctionTerminationStrategy(
            agents=[agent_reviewer],
            function=termination_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(result.value[0]).lower() == "yes",
            history_variable_name="history",
            maximum_iterations=10,
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(
                result.value[0]) if result.value is not None else FRONTDESK_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
    )

    user_input = "I would like to go to Paris."

    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))
    print(f"# User: '{user_input}'")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")

    print(f"# IS COMPLETE: {chat.is_complete}")

await main()

# User: 'I would like to go to Paris.'
# Agent - FrontDesk: 'Visit the Louvre Museum.'
# Agent - Concierge: 'While the Louvre Museum is certainly a world-renowned attraction, it's often bustling with tourists. For a more local experience, you might want to explore a less frequented gallery or neighborhood where you can immerse yourself in the local art scene and Parisian life. Consider areas that offer cultural experiences beyond the usual hotspots to truly connect with the essence of Paris.'
# Agent - FrontDesk: 'Visit the Louvre Museum.'
# Agent - Concierge: 'The Louvre Museum is an iconic destination, but if you're looking for a more authentic experience in Paris, I would suggest exploring beyond the typical tourist attractions. Seek out local neighborhoods where you can engage with Parisians and their way of life. This could be a quaint corner cafe or perhaps a market where locals frequent. Aim to find places with a rich sense of community and history.'
# Agent - FrontDesk: 'What o